In [1]:
#Imporing the important libraries such as pandas, numpy
import pandas as pd
import numpy as np
#Importing seaborn for ploting
import seaborn as sns
#Importing matplotlib for ploting
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.ensemble import RandomForestRegressor

import re

#Importing train_test_split to split the data to train and test
from sklearn.model_selection import train_test_split

#Importing countVectorizer to do vectorization
from sklearn.feature_extraction.text import CountVectorizer

#Importing model comparision metrics
from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# from scikitplot.metrics import plot_roc, plot_precision_recall

from sklearn.metrics import average_precision_score
#Importing LogisticRegression model
from sklearn.linear_model import LogisticRegression

#Importing the LinearSVC model
from sklearn.svm import LinearSVC
#Importing plot_roc, plot_precision_recall to plotting
# from scikitplot.metrics import plot_roc, plot_precision_recall
#Importing CalibratedClassifierCV
from sklearn.calibration import CalibratedClassifierCV

#Importing TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
#Importing RandomForestClassifier to build rf model
from sklearn.ensemble import RandomForestClassifier
#Importing AdaBoostClassifier model
from sklearn.ensemble import AdaBoostClassifier
#Improting Tokenizer to use it in the tokenization process
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#Importing tensorflow
import tensorflow as tf
#Importing optimization
# from official.nlp import optimization

#Importing wordCloud
from wordcloud import WordCloud

import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (18,6)
pd.set_option('display.max_columns', 50)

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
import pandas as pd

# Load the dataset from the URL or file
url = "https://github.com/hibahdesu/News-Sentiment-Analysis/raw/main/data/news.csv"
df = pd.read_csv(url)

# Convert 'date' column to datetime format for easier manipulation
df['date'] = pd.to_datetime(df['date'])

# Check the dataset structure
df.head()

,date,news,neg,neu,pos,compound,sentiment
0,2007-07-07,It was a long antipodean night. While there’s ...,0.059000,0.87800,0.064000,0.051600,POSITIVE
1,2007-07-07,In Mexico there are no licensing or registrati...,0.044000,0.95600,0.000000,-0.296000,NEGATIVE
2,2007-07-07,The government has until Monday to protect the...,0.000000,0.89400,0.106000,0.381800,POSITIVE
3,2007-07-07,A record-breaking heat wave in the Southwest i...,0.197505,0.66149,0.141005,0.997491,POSITIVE
4,2007-07-07,England started its Live Earth concert at Wemb...,0.033000,0.94500,0.022000,-0.177900,NEGATIVE


# Data Cleaning

In [4]:
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stopWords = set(stopwords.words("english"))

def cleanText(d):
    try:
        print("Starting text cleaning...")
        # 1- Tokenize and convert the words to lower case
        # tokens = word_tokenize(d.lower())  # Or use split() as fallback
        # print(f"Tokens: {tokens}")

        tokens = d.lower().split()  # This will split the text by whitespace

        print(f"Tokens: {tokens}")

        # 2- Removing puncs and numbers
        punc = [w for w in tokens if w.isalpha()]
        print(f"Alpha tokens: {punc}")

        # 3- Removing bad characters (URLs)
        d = re.sub('http\S*', '', d).strip()
        d = re.sub('www\S*', '', d).strip()

        # 4- Removing bad characters (Tags)
        d = re.sub('#\S*', '', d).strip()

        # 5- Removing bad characters (Mentions)
        d = re.sub('@\S*', '', d).strip()

        # 6- Removing upper brackets to keep negative auxiliary verbs in text
        d = d.replace("'", "")

        # 7- Removing Stopwords
        sw = [t for t in punc if t not in stopWords]

        # 8. Lemmatization
        cleaned = [WordNetLemmatizer().lemmatize(t) for t in sw]

        # 9. Joining the text with each other
        return " ".join(cleaned)
    except Exception as e:
        print(f"Error in cleanText: {e}")
        return None
#Creating the cleaned dataframe with the new function
cleaned = pd.DataFrame(df["news"].apply(cleanText))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Streaming output truncated to the last 5000 lines.
Tokens: ['the', 'conservative', 'party', 'says', 'it', 'would', 'review', 'licencing', 'laws', 'in', 'a', 'bid', 'to', 'boost', 'the', 'night-time', 'economy.']
Alpha tokens: ['the', 'conservative', 'party', 'says', 'it', 'would', 'review', 'licencing', 'laws', 'in', 'a', 'bid', 'to', 'boost', 'the']
Starting text cleaning...
Tokens: ['fiscal', 'challenges', 'labour', 'could', 'face', 'and', 'fallout', 'from', 'nigel', "farage's", 'ukraine', 'war', 'claims', 'lead', 'the', 'papers.']
Alpha tokens: ['fiscal', 'challenges', 'labour', 'could', 'face', 'and', 'fallout', 'from', 'nigel', 'ukraine', 'war', 'claims', 'lead', 'the']
Starting text cleaning...
Tokens: ['a', 'german', 'girl', 'organises', 'a', 'big', 'contribution', 'to', 'a', 'south', 'african', 'charity', 'tackling', 'period-poverty', 'issues.']
Alpha tokens: ['a', 'german', 'girl', 'organises', 'a', 'big', 'contribution', 'to', 'a', 'south', 'african', 'charity', 'tackling']
S

In [5]:
df["news"] = cleaned['news']
df["news"]

,news
0,long antipodean telling reflection national ta...
1,mexico licensing registration requirement real...
2,government monday protect redacted name baseba...
3,heat wave southwest pushing
4,england started live earth concert wembley sta...
...,...
427477,car set beer can stone thrown police demonstra...
427478,great britain continue successful start summer...
427479,pentagon announces middle east reinforcement i...
427480,bucharest lifted moratorium bear hunting popul...


# Processing and Splittin part

In [6]:
#Seed value
s = 101

In [7]:
#Splitting the dataset to X and y
X = df["news"] #Feature
y = df["compound"] #Target

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [9]:
#Spliting the X and y to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=s)

In [10]:
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

In [11]:
#Seeing the shape of y_train, and y_test
print('The shape of y_train: ', y_train_scaled.shape)
print('The shape of y_test: ', y_test_scaled.shape)

The shape of y_train:  (341985, 1)
The shape of y_test:  (85497, 1)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the TfidfVectorizer with the desired options
vectorizer = TfidfVectorizer(preprocessor=cleanText, min_df=3, max_features=10000, stop_words='english')  # Limit to 10,000 features

# Fit and transform the training data
X_train_tf = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_tf = vectorizer.transform(X_test)


Streaming output truncated to the last 5000 lines.
Tokens: ['voter', 'part', 'england', 'northern', 'ireland', 'able', 'elect', 'new', 'councillor']
Alpha tokens: ['voter', 'part', 'england', 'northern', 'ireland', 'able', 'elect', 'new', 'councillor']
Starting text cleaning...
Tokens: ['child', 'express', 'fandom', 'latest', 'parent', 'face', 'new', 'question', 'cultural', 'appropriation']
Alpha tokens: ['child', 'express', 'fandom', 'latest', 'parent', 'face', 'new', 'question', 'cultural', 'appropriation']
Starting text cleaning...
Tokens: ['eight', 'people', 'died', 'travis', 'astroworld', 'hundred', 'injured', 'investigator', 'sought', 'determine', 'surging', 'crowd', 'turned']
Alpha tokens: ['eight', 'people', 'died', 'travis', 'astroworld', 'hundred', 'injured', 'investigator', 'sought', 'determine', 'surging', 'crowd', 'turned']
Starting text cleaning...
Tokens: ['foreign', 'minister', 'say', 'russia', 'band', 'reflects', 'filming']
Alpha tokens: ['foreign', 'minister', 'say', 

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score


# Define the models
models = {
    # "Random Forest": RandomForestRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Support Vector Regressor": SVR(),
     "Random Forest": RandomForestRegressor(random_state=s),
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Train the model
    model.fit(X_train_tf, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_tf)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"{model_name} Results:")
    print(f"R2 Score: {r2}")
    print(f"Mean Squared Error: {mse}\n")


Training Linear Regression...
Linear Regression Results:
R2 Score: 0.5847125673322966
Mean Squared Error: 0.16120704058134402



## Sorting

In [14]:
df_sorted_n = df.sort_values(by="compound", ascending=True)
df_sorted_n

,date,news,neg,neu,pos,compound,sentiment
57180,2014-09-13,pope say many conflict amount piecemeal world ...,0.529000,0.471000,0.000000,-0.990800,NEGATIVE
26970,2011-05-23,war drug decreed million dollar govt failed cu...,0.465000,0.510000,0.025000,-0.989800,NEGATIVE
70782,2016-03-10,china strongly rejected criticism human right ...,0.489000,0.445000,0.066000,-0.985400,NEGATIVE
182524,2018-04-25,marshal view job critical protecting plane air...,0.556000,0.444000,0.000000,-0.984700,NEGATIVE
47740,2013-08-31,leader cannibal rape cult hacked death village...,0.470000,0.489000,0.041000,-0.984700,NEGATIVE
...,...,...,...,...,...,...,...
93213,2016-10-19,absorbing solo show eerily timely offering gif...,0.001952,0.036170,0.961878,0.999880,POSITIVE
7032,2009-03-15,right join israel,0.235976,0.501165,0.262859,0.999880,POSITIVE
104298,2016-12-21,classic directed bartlett timely shine close new,0.000787,0.048695,0.950518,0.999881,POSITIVE
90765,2016-10-05,captivating gem give green,0.008067,0.405381,0.586552,0.999886,POSITIVE


In [15]:
df_sorted_p = df.sort_values(by="compound", ascending=False)
df_sorted_p

,date,news,neg,neu,pos,compound,sentiment
354531,2021-06-09,heritage commodity variety,0.003685,0.108553,0.887761,0.999887,POSITIVE
90765,2016-10-05,captivating gem give green,0.008067,0.405381,0.586552,0.999886,POSITIVE
104298,2016-12-21,classic directed bartlett timely shine close new,0.000787,0.048695,0.950518,0.999881,POSITIVE
7032,2009-03-15,right join israel,0.235976,0.501165,0.262859,0.999880,POSITIVE
93213,2016-10-19,absorbing solo show eerily timely offering gif...,0.001952,0.036170,0.961878,0.999880,POSITIVE
...,...,...,...,...,...,...,...
182524,2018-04-25,marshal view job critical protecting plane air...,0.556000,0.444000,0.000000,-0.984700,NEGATIVE
47740,2013-08-31,leader cannibal rape cult hacked death village...,0.470000,0.489000,0.041000,-0.984700,NEGATIVE
70782,2016-03-10,china strongly rejected criticism human right ...,0.489000,0.445000,0.066000,-0.985400,NEGATIVE
26970,2011-05-23,war drug decreed million dollar govt failed cu...,0.465000,0.510000,0.025000,-0.989800,NEGATIVE
